# Introduction to Subject-Specific Vagus Scaffolds

Subject-Specific Vagus (SSV) Scaffolds created for the SPARC REVA Phase 2 project are a specialized Finite Element Model which defines a consistent coordinate system for the volume of the human vagus trunk, with arbitrary numbers of branches radiating out from starting locations embedded in the trunk/parent branch volume. Each scaffold is defined with three coordinate fields:

1. `coordinates` fitted to the geometry the scaffold was fitted, usually the excised configuration stitched from micro-CT segmentations. The trunk also includes fitted orientation/twist and radius where data has been provided. Branches are fitted to provided path data.
2. `straight coordinates`, a version of the geometric coordinates using the same scale but with the trunk going straight down the z-axis, and all twist removed so that the x-axis is anatomically left and y-axis is the anterior direction, centred at x, y = (0, 0). Branches radiate out in their starting anatomical directions.
3. `vagus coordinates`, an invariant 'material' coordinate system for the trunk of all subjects, with the z/3-axis giving a normalized measure down the vagus from 0.0 near the brainstem to 1.0 at full length but actually calibrated by level markers at fixed locations down its centroid. These coordinates have fixed radius giving an aspect ratio equivalent to 3mm diameter of nerve epineurium over 500mm length.

![SSV coordinate fields](images/ssv_3_coordinate_fields.png "Subject-Specific Vagus Scaffolds, from left-to-right 1. coordinates, 2. straight coordinates, 3. vagus coordinates (unit length)")
**Figure 1. Subject-Specific Vagus Scaffold coordinates** From left-to-right: 1. coordinates, 2. straight coordinates, 3. vagus coordinates (scaled from unit length). Translucent surfaces show the round equivalent-area epineurium surface, but the bounds of the scaffold is a box of twice these dimensions to enclose the true cross-section shape and nearby tissue.

The boxes along the vagus trunk and branches in Figure 1 are 3-D hexahedral finite elements with local coordinate systems spanning from 0.0 to 1.0 along three directions down the vagus, left and up. These are entirely defined as functions of coordinate position and direction parameters along the centroid of the trunk and branches, and 2-D epineurium surfaces and 1-D centroid lines are defined from the same parameters. The start of each branch is defined  by general linear mapping of parent/trunk parameters making them fully embedded or built-in to their host at that point, preserving their relative directions when the scaffold is fitted to other configurations such as in-body trace data.

Due to the specialized nature of these models and the esoteric EX file format it is easiest to extract information by evaluating it using the [cmlibs.zinc library API](https://cmlibs.org/documentation/api), or **Zinc** from its Python interface. This can be installed via pip:

In [1]:
pip install cmlibs.zinc

Note: you may need to restart the kernel to use updated packages.


The following code reads a simple test vagus scaffold included with this tutorial (noting that one can equivalently load the EX file output from the Scaffold Creator plug produced from stitched vagus micro-CT segmentations in the REVA-derived vagus scaffold dataset), gets the trunk group, the coordinate fields and evaluates the geometric coordinates at locations down the centroid of the trunk.

In [6]:
from cmlibs.zinc.context import Context
from cmlibs.zinc.result import RESULT_OK

# the Zinc Context is the top-level object from which all others are obtained
context = Context("SSV introduction")
# a region is like a file system folder, in which meshes, nodeset and fields are equivalent to files, and child regions have their own namespaces
region = context.getDefaultRegion()
result = region.readFile("resources/vagus_test_scaffold1.exf")
print("Read scaffold success:", result == RESULT_
# the fieldmodule provides API for accessing objects local to the region
fieldmodule = region.getFieldmodule()
# the following fields are defined on the scaffold as shown above
coordinates = fieldmodule.findFieldByName("coordinates")
straight_coordinates = fieldmodule.findFieldByName("straight coordinates")
vagus_coordinates = fieldmodule.findFieldByName("vagus coordinates")
# each of these fields produces 3 components
for field in (coordinates, straight_coordinates, vagus_coordinates):
    print(field.getName(), "has", coordinates.getNumberOfComponents(), "components.")
# in order to evaluate fields we need a Fieldcache object in which we specify where to evaluate, and in which intermediate values are cached
fieldcache = fieldmodule.createFieldcache()
# a mesh is a collection of all the elements of a given dimension in the region
mesh3d = fieldmodule.findMeshByDimension(3)
# a group is dually a container of model parts such as elements and node, and a field returning True at locations within its domain
# we need to cast it to it's derived type to access type-specific API:
trunk_group = fieldmodule.findFieldByName("left vagus nerve").castGroup()
# a mesh group is a specialization of a mesh containing only the subset of elements from that mesh which have been added to this group
trunk_mesh_group3d = trunk_group.getMeshGroup(mesh3d)
print("Vagus trunk contains", trunk_mesh_group3d.getSize(), "elements.")
# we can iterate over the mesh or the mesh group; in either case iteration is in the same order as the element identifiers (i.e. numbers)
elementiterator = trunk_mesh_group3d.createElementiterator()
# following is the location in each element to evaluate: 1/4 along (down the vagus) and at the centroid of the cross-section
xi = [0.25, 0.5, 0.5]
element = elementiterator.next()
while element.isValid():
    element_identifier = element.getIdentifier()
    fieldcache.setMeshLocation(element, xi)
    result, x = coordinates.evaluateReal(fieldcache, 3)
    print("Element", element_identifier, "coordinates", x)
    element = elementiterator.next()

Read scaffold success: True
coordinates has 3 components.
straight coordinates has 3 components.
vagus coordinates has 3 components.
Vagus trunk contains 25 elements.
Element 1 coordinates [-720.8203637680142, -6652.227292847368, -38.10529956217343]
Element 2 coordinates [1906.8389128225303, -7374.81394841826, 50.45597229581807]
Element 3 coordinates [4268.839102884901, -6550.804058621278, -4.619454469083853]
Element 4 coordinates [5896.570479428751, -4546.896355884268, -171.34989651102575]
Element 5 coordinates [7933.219021121138, -3226.798516228627, -303.1244298308453]
Element 6 coordinates [10262.278713530744, -2701.1377344879293, -388.2400323890354]
Element 7 coordinates [12653.932997043496, -2671.4261721152957, -444.3926826701179]
Element 8 coordinates [15090.476107135917, -2687.239397013903, -449.51294659698925]
Element 9 coordinates [17528.473750782807, -2677.898895464176, -432.29371373631153]
Element 10 coordinates [19970.522026089853, -2841.3200970070307, -573.6343430536429]
E

The following code gets a branch group and determines the vagus coordinates where it originates in the parent (trunk), and the anatomical direction it initially points

In [15]:
# get function to make a vector init length
from cmlibs.maths.vectorops import normalize

branch_name = "left A thoracic cardiopulmonary branch of vagus nerve"
branch_group = fieldmodule.findFieldByName(branch_name).castGroup()
print("Found branch", branch_group.isValid())
branch_mesh_group3d = branch_group.getMeshGroup(mesh3d)
# get the first element in the branch, since these are created from the start with increasing element numbers
first_branch_element = branch_mesh_group3d.createElementiterator().next()
# want to evaluate the location and direction at the very start, i.e. 0.0 along the first branch element
xi = [0.0, 0.5, 0.5]
# need to request derivative of coordinates w.r.t. element coordinate along the vagus
derivative_xi1 = mesh3d.getChartDifferentialoperator(1, 1)
fieldcache.setMeshLocation(first_branch_element, xi)
_, branch_vagus_coordinate = vagus_coordinates.evaluateReal(fieldcache, 3)
_, d1 = straight_coordinates.evaluateDerivative(derivative_xi1, fieldcache, 3)
# branch direction axes are (left, anterior, down the body)
branch_direction = normalize(d1)
print("Branch '" + branch_name + "':\n    vagus coordinates", branch_vagus_coordinate, "\n    anatomical direction", branch_direction)

Found branch True
Branch 'left A thoracic cardiopulmonary branch of vagus nerve':
    vagus coordinates [-0.0003328408968624165, 1.7493737182796838e-05, 0.3813658363063589] 
    anatomical direction [-0.9097938227559508, 0.38257762513656673, 0.16096447067591738]


This ends this first introduction which demonstrates the type of code to query and evaluate information in vagus scaffolds.

Subsequent tutorials will increasingly use specially-written utility functions to extract key information without forcing the use of low-level coding. All these utilities will be free and open source, but that doesn't mean they will be trivial to understand; the scaffolds are encoded as finite element models, with generic grouping features used to mark branches and other features. It's often necessary to do extra work to recover hierarchical groupings which are not inherently understood in the generic structure.